In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from modules import io
from modules import vascular_data as sv

In [ ]:
g_cfg_fn   = "./config/global.yaml"
c_cfg_fn   = "./config/fcnet_svd_mr.yaml"

global_config = load_yaml(g_cfg_fn)
case_config   = load_yaml(c_cfg_fn)

FILES         = read_lines(case_config['FILE_LIST'])
print(FILES[:2])

train_id_list = [i for i in FILES if any([c in i for c in case_config['TRAIN_PATTERNS']])]
val_id_list   = [i for i in FILES if any([c in i for c in case_config['VAL_PATTERNS']])]
test_id_list  = [i for i in FILES if any([c in i for c in case_config['TEST_PATTERNS']])]